# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Import API key
geoapify_key = "af4166e762514b0785bce184f557dbf6"

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san pedro pochutla,15.7333,-96.4667,87.51,86,100,6.55,MX,1718397004
1,1,port-aux-francais,-49.3500,70.2167,42.78,91,100,31.05,TF,1718397005
2,2,taulaga,-11.0553,-171.0883,83.71,78,100,15.10,AS,1718397005
3,3,al mijlad,11.0333,27.7333,86.81,43,100,7.56,SD,1718397006
4,4,touros,-5.1989,-35.4608,77.54,86,72,9.31,BR,1718397007


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', 
    color='Humidity', alpha=0.6, frame_width=800, frame_height=600, 
    title="City Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 20)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,campos belos,-13.0367,-46.7717,77.54,36,0,2.91,BR,1718397012
75,75,nova vicosa,-17.8919,-39.3719,74.01,74,0,6.60,BR,1718397023
80,80,jamestown,42.0970,-79.2353,74.84,38,0,4.61,US,1718396731
92,92,prado,-17.3411,-39.2208,74.50,79,0,6.80,BR,1718397027
98,98,bethel,41.3712,-73.4140,78.03,79,0,0.00,US,1718397028


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
31,campos belos,BR,-13.0367,-46.7717,36,
75,nova vicosa,BR,-17.8919,-39.3719,74,
80,jamestown,US,42.0970,-79.2353,38,
92,prado,BR,-17.3411,-39.2208,79,
98,bethel,US,41.3712,-73.4140,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 5000  # e.g., 5000 meters
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,  # to get the nearest hotel
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
campos belos - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
jamestown - nearest hotel: DoubleTree Jamestown
prado - nearest hotel: Pousada Recanto do Prado
bethel - nearest hotel: Hampton Inn Danbury
north bend - nearest hotel: The Mill Casino Hotel
zhanaozen - nearest hotel: No hotel found
altamont - nearest hotel: Quality Inn
tongren - nearest hotel: 美高酒店
nurota - nearest hotel: Nurota
alto piquiri - nearest hotel: No hotel found
don luan - nearest hotel: Thanh Sang
cruzeiro do sul - nearest hotel: Estrela Palace Hotel
gharavuti - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
troitsk - nearest hotel: Амур
kencong - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
varzelandia - nearest hotel: No hotel found
wuhu - nearest hotel: Crowne Plaza Hotel Wuhu
saint-pierre - nearest hotel: Tropic Hotel
kupang - nearest hotel: HOTEL SALUNGA
'ain el hadjar - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
31,campos belos,BR,-13.0367,-46.7717,36,No hotel found
75,nova vicosa,BR,-17.8919,-39.3719,74,Pousada Pontal da Barra
80,jamestown,US,42.0970,-79.2353,38,DoubleTree Jamestown
92,prado,BR,-17.3411,-39.2208,79,Pousada Recanto do Prado
98,bethel,US,41.3712,-73.4140,79,Hampton Inn Danbury


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

hotel_df['Hover'] = hotel_df['City'] + ", " + hotel_df['Country'] + "\nHotel: " + hotel_df['Hotel Name']

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, size="Humidity", color="Humidity", 
    alpha=0.5, hover_cols=["Hover"], 
    tiles="OSM"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover)